In [ ]:
library(Biostrings)
library(lessR)
library(vegan)
library(phyloseq)
library(ggplot2)
library(ape)
library(picante)
library(ggpubr)
library(readxl)
library(devtools)
library(pairwiseAdonis)
library(compositions)
library(microbiome)
library(readxl)
library(dbplyr)
library(dplyr)


setwd("~/Documents/analysis/")
path <- "~/Documents/analysis/"


#Import data into phyloseq
tree <- read.nexus(file="2038.tre")
class(tree)

#import taxa
all_taxa_table<- as.matrix(read.csv("hannataxa.csv", 
                                    sep=",", header=T, row.names=1))
dim(all_taxa_table)
otu <- as.matrix(read.csv("hanna_counts.csv", 
                           sep=",", header=T, row.names=1))
dim(otu)

All_seqs <- readDNAStringSet("All_tab_nochim.fa",format = "fasta")
length(All_seqs)

OTU = otu_table(otu, taxa_are_rows = TRUE)
dim(OTU)

metadata11 <- read_excel("metadata11.xlsx")
#View(metadata11)
meta <- as.data.frame(metadata11)
rownames(meta) <- meta$GroupID
dim(meta)
#View(meta)

All_samples_phylo <- phyloseq(otu_table(OTU), 
                              tax_table(all_taxa_table), 
                              phy_tree(tree), refseq(All_seqs), 
                              sample_data(meta))
All_samples_phylo
All_samples_phylo1 <- subset_taxa(All_samples_phylo, !is.na(Phylum) & !Phylum %in% c("", "uncharacterized"))
All_samples_phylo1
#1898 ASVs
All_samples_phylo1@sam_data$julian <- as.numeric(as.character(All_samples_phylo1@sam_data$julian))
All_samples_phylo1@sam_data$Reduced <- as.numeric(as.character(All_samples_phylo1@sam_data$Reduced))
All_samples_phylo1@sam_data$pairedReads <- as.numeric(as.character(All_samples_phylo1@sam_data$pairedReads))

TAXe = as(tax_table(All_samples_phylo1), "matrix")
write.csv(TAXe, "correct_tax.csv" )

# Extract abundance matrix from the phyloseq object
OTU1 = as(otu_table(All_samples_phylo1), "matrix")
# transpose if necessary
if(taxa_are_rows(All_samples_phylo1)){OTU1 <- t(OTU1)}
# Coerce to data.frame
OTUdf = as.data.frame(OTU1)
View(OTUdf)
dim(OTUdf)
write.csv(file = "OTU_reduced_all.csv", t(OTUdf))



In [ ]:
#### pruning samples with low counts
pruned_samples <- c("hdmarch_2018_339",
                    "hdmarch_2018_343",
                    "hdmarch_2018_98",
                    "hdmarch_2018_86",
                    "hdmarch_2018_84",
                    "hdmarch_2018_5",
                    "hdmarch_2018_46",
                    "hdmarch_2018_35",
                    "hdmarch_2018_59",
                    "hdmarch_2018_48",
                    "hdmarch_2018_25",
                    "hdmarch_2018_224",
                    "hdmarch_2018_263",
                    "hdmarch_2018_81",
                    "hdmarch_2018_369",
                    "hdmarch_2018_259",
                    "hdmarch_2018_82",
                    "hdmarch_2018_307",
                    "hdmarch_2018_83",
                    "hdmarch_2018_336",
                    "hdmarch_2018_261",
                    "hdmarch_2018_234",
                    "hdmarch_2018_296",
                    "hdmarch_2018_229",
                    "hdmarch_2018_214",
                    "hdmarch_2018_309",
                    "hdmarch_2018_329",
                    "hdmarch_2018_80",
                    "hdmarch_2018_274",
                    "hdmarch_2018_249",
                    "hdmarch_2018_271",
                    "hdmarch_2018_273",
                    "hdmarch_2018_250",
                    "hdmarch_2018_294",
                    "hdmarch_2018_228",
                    "hdmarch_2018_254",
                    "hdmarch_2018_210",
                    "hdmarch_2018_90",
                    "hdmarch_2018_376",
                    "hdmarch_2018_295",
                    "hdmarch_2018_278",
                    "hdmarch_2018_332",
                    "hdmarch_2018_297",
                    "hdmarch_2018_272",
                    "hdmarch_2018_258",
                    "hdmarch_2018_298",
                    "hdmarch_2018_360",
                    "hdmarch_2018_262",
                    "hdmarch_2018_238",
                    "hdmarch_2018_264",
                    "hdmarch_2018_240",
                    "hdmarch_2018_78",
                    "hdmarch_2018_76",
                    "hdmarch_2018_222",
                    "hdmarch_2018_79",
                    "hdmarch_2018_204",
                    "hdmarch_2018_227",
                    "hdmarch_2018_231",
                    "hdmarch_2018_72",
                    "hdmarch_2018_75",
                    "hdmarch_2018_71",
                    "hdmarch_2018_36",
                    "hdmarch_2018_51",
                    "hdmarch_2018_50",
                    "hdmarch_2018_66",
                    "hdmarch_2018_40",
                    "hdmarch_2018_22",
                    "hdmarch_2018_30",
                    "hdmarch_2018_91",
                    "hdmarch_2018_26",
                    "hdmarch_2018_4",
                    "hdmarch_2018_24",
                    "hdmarch_2018_61",
                    "hdmarch_2018_64",
                    "hdmarch_2018_23",
                    "hdmarch_2018_7",
                    "hdmarch_2018_3",
                    "hdmarch_2018_223",
                    "hdmarch_2018_34",
                    "hdmarch_2018_47",
                    "hdmarch_2018_27",
                    "hdmarch_2018_28",
                    "hdmarch_2018_275",
                    "hdmarch_2018_33",
                    "hdmarch_2018_52",
                    "hdmarch_2018_280",
                    "hdmarch_2018_38",
                    "hdmarch_2018_29",
                    "hdmarch_2018_44",
                    "hdmarch_2018_60",
                    "hdmarch_2018_221",
                    "hdmarch_2018_370",
                    "hdmarch_2018_225",
                    "hdmarch_2018_208",
                    "hdmarch_2018_251",
                    "hdmarch_2018_203",
                    "hdmarch_2018_85",
                    "hdmarch_2018_119",
                    "hdmarch_2018_186",
                    "hdmarch_2018_151",
                    "hdmarch_2018_121",
                    "hdmarch_2018_143",
                    "hdmarch_2018_19",
                    "hdmarch_2018_200",
                    "hdmarch_2018_109",
                    "hdmarch_2018_116",
                    "hdmarch_2018_155",
                    "hdmarch_2018_164",
                    "hdmarch_2018_190",
                    "hdmarch_2018_146",
                    "hdmarch_2018_167",
                    "hdmarch_2018_108",
                    "hdmarch_2018_189",
                    "hdmarch_2018_154",
                    "hdmarch_2018_127",
                    "hdmarch_2018_145",
                    "hdmarch_2018_191",
                    "hdmarch_2018_141",
                    "hdmarch_2018_106",
                    "hdmarch_2018_152",
                    "hdmarch_2018_149",
                    "hdmarch_2018_114",
                    "hdmarch_2018_137",
                    "hdmarch_2018_201",
                    "hdmarch_2018_182",
                    "hdmarch_2018_138",
                    "hdmarch_2018_150",
                    "hdmarch_2018_110",
                    "hdmarch_2018_179",
                    "hdmarch_2018_126",
                    "hdmarch_2018_104",
                    "hdmarch_2018_177",
                    "hdmarch_2018_100",
                    "hdmarch_2018_133",
                    "hdmarch_2018_101",
                    "hdmarch_2018_148",
                    "hdmarch_2018_130",
                    "hdmarch_2018_123",
                    "hdmarch_2018_125",
                    "hdmarch_2018_1",
                    "hdmarch_2018_16",
                    "hdmarch_2018_12",
                    "hdmarch_2018_17",
                    "hdmarch_2018_11",
                    "hdmarch_2018_20",
                    "hdmarch_2018_2",
                    "hdmarch_2018_18",
                    "hdmarch_2018_10",
                    "hdmarch_2018_124",
                    "hdmarch_2018_142")
length(pruned_samples)
pruned_samples_phylo <- prune_samples(pruned_samples, All_samples_phylo1)
pruned_samples_phylo

In [ ]:
#get otu df from pruned samples phylo
OTUe = as(otu_table(pruned_samples_phylo), "matrix")
# transpose if necessary
if(taxa_are_rows(pruned_samples_phylo)){OTUe <- t(OTUe)}
# Coerce to data.frame
OTUee = as.data.frame(OTUe)
#View(OTUee)
dim(OTUee)
#View(pruned_samples_phylo@tax_table)
TAXee = as(tax_table(pruned_samples_phylo), "matrix")
#View(TAXee)
#remove rare taxa
pruned_samples_phylo1 <- prune_taxa(taxa_sums(pruned_samples_phylo) > 1, pruned_samples_phylo)
pruned_samples_phylo1

#get metadata table
length(sample_names(pruned_samples_phylo1))
pruned_samples_meta <- as.matrix(pruned_samples_phylo1@sam_data, stringsAsFactors=T)
pruned_samples_meta <- as.data.frame(pruned_samples_meta, stringsAsFactors=T)
pruned_samples_meta$pairedReads <- as.numeric(as.character(pruned_samples_meta$pairedReads))
pruned_samples_meta$Reduced <- as.numeric(as.character(pruned_samples_meta$Reduced))
pruned_samples_meta$julian <- as.numeric(as.character(pruned_samples_meta$julian))
dim(pruned_samples_meta)
#View(pruned_samples_meta)
sapply(pruned_samples_meta, class)

In [ ]:
#transform
clr_phylo <- transform(pruned_samples_phylo1, transform = "clr", target = "sample")
clr_phylo
clr_phylo_otu <- as.data.frame(clr_phylo@otu_table)
clr_phylo

In [ ]:
#get distance
t_clr_phylo_otu <- t(clr_phylo_otu)
clr_euc_all <- vegdist(t_clr_phylo_otu, method = "euclidean")
clr_euc_all
clr_phylo_meta <- as.matrix(clr_phylo@sam_data, stringsAsFactors=T)
clr_phylo_meta <- as.data.frame(clr_phylo_meta, stringsAsFactors=T)
clr_phylo_meta$pairedReads <- as.numeric(as.character(clr_phylo_meta$pairedReads))
clr_phylo_meta$Reduced <- as.numeric(as.character(clr_phylo_meta$Reduced))
clr_phylo_meta$julian <- as.numeric(as.character(clr_phylo_meta$julian))
dim(clr_phylo_meta)
View(clr_phylo_meta)
sapply(clr_phylo_meta, class)

In [ ]:
#tests
v <- betadisper(clr_euc_all, clr_phylo_meta$sample_type, type = c("median","centroid"))
permutest(v)
#yes
adonis2(clr_euc_all ~ pairedReads, data = clr_phylo_meta)
#sig
adonis2(clr_euc_all ~ Reduced, data = clr_phylo_meta)
#sig
adonis2(clr_euc_all ~ sample_type, data = clr_phylo_meta)
adonis2(clr_euc_all ~ fraction, data = clr_phylo_meta)
adonis2(clr_euc_all ~ sample_type * pairedReads, data = clr_phylo_meta)

In [ ]:
#reads by group and compare
clr_phylo_meta2$group <- ordered(clr_phylo_meta2$sample_type,
                         levels = c("Algae", "Raw Intake", "Treated Intake", "Waste", "Larvae Tank"))

group_by(clr_phylo_meta2, group) %>%
  summarise(
    count = n(),
    mean = mean(pairedReads, na.rm = TRUE),
    sd = sd(pairedReads, na.rm = TRUE),
    median = median(pairedReads, na.rm = TRUE),
    IQR = IQR(pairedReads, na.rm = TRUE)
  )

View(meta)
?aov
t <- aov(data = meta, Reduced ~ sample_type)
class(t)
summary(t)

library("ggpubr")
ggboxplot(clr_phylo_meta2, x = "group", y = "Reduced", 
          color = "group",
          order = c("Algae", "Raw Intake", "Treated Intake", "Waste", "Larvae Tank"),
          ylab = "Reduced", xlab = "sample type")
kruskal.test(Reduced ~ group, data = clr_phylo_meta2)
#not sig
#sig for paired reads 
pairwise.wilcox.test(clr_phylo_meta2$pairedReads, clr_phylo_meta2$group,
                     p.adjust.method = "BH")
#sig difference in intake vs algae and treated vs larvae tank 

In [ ]:
#ordinate (entire group)
#PCA via phyloseq
OK <- ordinate(clr_phylo, "RDA", "euclidean")
OK
#Plot scree plot
plot_scree(OK) + geom_bar(stat="identity", fill = "blue") +
  labs(x = "\nAxis", y = "Proportion of Variance\n")
#Examine eigenvalues and % prop. variance explained
head(OK$CA$eig)   
sapply(OK$CA$eig[1:5], function(x) x / sum(OK$CA$eig)) 
#Scale axes and plot ordination
clr1 <- OK$CA$eig[1] / sum(OK$CA$eig)
clr2 <- OK$CA$eig[2] / sum(OK$CA$eig)

?scores
pl <- as.data.frame(scores(OK, display = "sites"))
pll <- as.data.frame(pl)
View(pll)

hanPalette1 <- c("chartreuse4", "goldenrod1", "royalblue", "lightskyblue",  "thistle4")


p2 <- plot_ordination(clr_phylo, OK, "samples", color="sample_type", shape = "fraction", axes = c(1, 2)) +
  coord_fixed(clr2 / clr1) +
  geom_point(color = "black", size=2.6) +  
  geom_point(size = 2, alpha = 0.95) +
  scale_color_manual(values = hanPalette1) +
  theme_bw() + labs(fill = "Day") +
  labs(title="Aquaculture Facility, all samples") + 
  theme(legend.text =element_text(size=6)) +
  theme(legend.title = element_text(size=6)) +
  theme(axis.title.x = element_text(size = 8)) +
  theme(axis.title.y = element_text(size = 8)) +
  theme(axis.text.x = element_text(size =6)) +
  theme(axis.text.y = element_text(size =6)) + theme(plot.title = element_text(hjust = 0.5)) 
p2 



In [ ]:
#Free-living - All
fl_phy <- pruned_samples_phylo1 %>% 
  phyloseq::subset_samples(fraction %in% "Supor 200") 
fl_phy2 <- fl_phy %>% 
  phyloseq::subset_samples(sample_type %in% c("Larvae Tank", "Treated Intake", "Raw Intake", "Algae", 'Waste')) 
fl_phy2
#get ASV table 
OTUe = as(otu_table(fl_phy2), "matrix")
# transpose if necessary
if(taxa_are_rows(fl_phy2)){OTUe <- t(OTUe)}
# Coerce to data.frame
OTUee = as.data.frame(OTUe)
View(OTUee)
dim(OTUee)
TAXee = as(tax_table(fl_phy2), "matrix")
#Revmove ASVs with no presence
fl_phy3 <- prune_taxa(taxa_sums(fl_phy2) > 1, fl_phy2)
#transform
fl_clr <- transform(fl_phy3, transform = "clr", target = "sample")
fl_phylo_otu <- as.data.frame(fl_phylo@otu_table)
dim(fl_phylo_otu)
View(fl_phylo_otu)
#get metadata table
meta <- as.matrix(fl_phylo@sam_data, stringsAsFactors=T)
meta <- as.data.frame(meta, stringsAsFactors=T)
meta$pairedReads <- as.numeric(as.character(meta$pairedReads))
meta$Reduced <- as.numeric(as.character(meta$Reduced))
meta$julian <- as.numeric(as.character(meta$julian))
dim(meta)
View(meta)
#View(pruned_samples_meta)
sapply(meta, class)


In [ ]:
##What if I subsetted the CLR transformed phyloseq object
clr_phylo_pa <- clr_phylo %>% 
  phyloseq::subset_samples(fraction %in% "GFF") 
clr_phylo_pa
clr_phylo_pa_a <- clr_phylo_pa %>% 
  phyloseq::subset_samples(Type2 %in% c("Tank", "Intake")) 
clr_phylo_pa_a
#get ASV table 
OTUe = as(otu_table(clr_phylo_pa_a), "matrix")
# transpose if necessary
if(taxa_are_rows(clr_phylo_pa_a)){OTUe <- t(OTUe)}
# Coerce to data.frame
OTUee = as.data.frame(OTUe)
View(OTUee)
dim(OTUee)
TAXee = as(tax_table(clr_phylo_pa_a), "matrix")
#Revmove ASVs with no presence
clr_phylo_pa_b <- prune_taxa(taxa_sums(clr_phylo_pa_a) > 1, clr_phylo_pa_a)

#get metadata table
meta <- as.matrix(clr_phylo_pa_b@sam_data, stringsAsFactors=T)
meta <- as.data.frame(meta, stringsAsFactors=T)
meta$pairedReads <- as.numeric(as.character(meta$pairedReads))
meta$Reduced <- as.numeric(as.character(meta$Reduced))
meta$julian <- as.numeric(as.character(meta$julian))
dim(meta)
View(meta)
#View(pruned_samples_meta)
sapply(meta, class)

#distance
clr_phylo_pa_b_otu <- as.data.frame(clr_phylo_pa_b@otu_table)
clr_phylo_pa_b_otu_t <- t(clr_phylo_pa_b_otu)
euc_pa <- vegdist(clr_phylo_pa_b_otu_t, method = "euclidean")
#tests


adonis2(euc_pa ~ sample_type, data = meta)
adonis2(euc_pa ~ Type2, data = meta)
adonis2(euc_pa ~ Type, data = meta)